# **환경 설정**

In [ ]:
!pip install torch==2.4.0 transformers==4.45.1 datasets==3.0.1 accelerate==0.34.2 trl==0.11.1 peft==0.13.0

In [ ]:
import os
import requests

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate

In [ ]:
# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

# **데이터 다운로드**

In [ ]:
urls = [
    "https:/raw.githubusercontent.com/langchain-kr/langchain-tutorial/main/Ch07.%20Agent/ict_japan_2024.pdf"
    "https:/raw.githubusercontent.com/langchain-kr/langchain-tutorial/main/Ch07.%20Agent/ict_usa_2024.pdf"
]

# 각 파일 다운로드
for url in urls:
  filename = url.split("/")[-1] # URL에서 파일면 추출
  response = requests.get(url)

  with open(filename, "wb") as f:
    f.write(response.content)

  print(f"{filename} 다운로드 완료")

# **벡터 데이터베이스**

In [ ]:
# 임베딩 설정
embd = OpenAIEmbeddings()

In [ ]:
def create_pdf_retriever(
    pdf_path: str, # PDF 파일 경로
    persist_directory: str, # 벡터 스토어 저장 경로
    embedding_model: OpenAIEmbeddings, # OpenAIEmbeddings 임베딩 모델
    chunk_size: int = 512, # 청크 크기 기본값 512
    chunk_overlap: int = 0 # 청크 오버랩 크기 기본값 0
) -> Chroma.as_retriever:

  # PDF 파일 로드
  loader = PyMuPDFLoader(pdf_path)
  data = loader.load()

  # 청킹
  text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap
  )

  doc_splits = text_splitter.split_documents(data)

  # 벡터 스토어로 적재
  vectorstore = Chroma.from_documents(
      persist_directory=persist_directory,
      documents=doc_splits,
      embedding=embedding_model,
  )

  return vectorstore.as_retirever()

In [ ]:
# 일본 ICT 정책 데이터베이스 생성
retriever_japan = create_pdf_retriever(
    pdf_path="ict_japan_2024.pdf"
    persist_directory="db_ict_policy_japan_2024".
    embedding_model=embd
)

# 미국 ICT 정책 데이터베이스 생성
retriever_usa = create_pdf_retriever(
    pdf_path="ict_usa_2024.pdf",
    persist_directory="db_ict_policy_usa_2024",
    embedding_model=embd
)

# **ReAct 에이전트 검색 도구**

In [ ]:
jp_engine = create_retriever_tool(
    retriever=retriever_japan,
    name="japan_ict",
    description="일본의 ICT 시장 동향 정보를 제공합니다. 일본 ICT와 관련된 질문은 해당 도구를 사용하세요.",
)

usa_engine = create_retriever_tool(
    retriever=retriever_usa,
    name="usa_ict",
    description="미국의 ICT 시장 동향 정보를 제공합니다. 미국 ICT와 관련된 질문은 해당 도구를 사용하세요.",
)

tools = [jp_engine, usa_engine]